In [ ]:
from dwave.system import FixedEmbeddingComposite, DWaveSampler
import numpy as np
import pickle


local_dir = "your local directory to save results"
token = "your d-wave token"
hamiltonian = "data_5.pkl"

data = []
with open(f"embedding_files/{hamiltonian}", 'rb') as fr:
    try:
        while True:
            data.append(pickle.load(fr))
    except EOFError:
        pass

num_reads = 2000
annealing_time = 0.02 #fast regime

dwave_hw = data[0]['solver']
qpu = DWaveSampler(solver=dwave_hw, token=token)

for instance in data:
    sampler_fixed = FixedEmbeddingComposite(qpu, instance["embedding"]) 
    sampleset = sampler_fixed.sample_ising(h={}, J=instance["J"], 
                                           num_reads=num_reads, 
                                           annealing_time=annealing_time,
                                           fast_anneal=True,
                                           auto_scale=False, 
                                           answer_mode="raw",
                                           flux_biases=instance["h"],
                                           chain_strength=2 
                                        )
    sampleset_qa = sampleset.aggregate()
    
    # Save results into local directory
    result = {'sampleset': sampleset_qa}
    with open(f"{local_dir}/results.pkl", 'ab') as f:
        pickle.dump(result, f)